In [1]:
#Installing libraries
!pip install pandas
!pip install pymongo
!pip install boto3

In [2]:
#Importing libraries to ingest data
import numpy as np
import pandas as pd
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO

# Data extraction and cleansing

In [8]:
#Reading file with states data
states = pd.read_csv('arquivos/estados.csv')
states

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte
5,16,AP,Amapá,1.41,-51.77,Norte
6,17,TO,Tocantins,-9.46,-48.26,Norte
7,21,MA,Maranhão,-5.42,-45.44,Nordeste
8,22,PI,Piauí,-6.60,-42.28,Nordeste
9,23,CE,Ceará,-5.20,-39.53,Nordeste


In [9]:
#Reading file with cities data
cities = pd.read_csv('arquivos/municipios.csv')
cities

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [10]:
#Joining states info with cities info
df = states.join(cities.set_index('codigo_uf'), on='codigo_uf', how='left', lsuffix='_state', rsuffix='_city')
df

,codigo_uf,uf,nome_state,latitude_state,longitude_state,regiao,codigo_ibge,nome_city,latitude_city,longitude_city,capital,siafi_id,ddd,fuso_horario
0,11,RO,Rondônia,-10.83,-63.34,Norte,1100015,Alta Floresta D'Oeste,-11.92830,-61.9953,0,33,69,America/Porto_Velho
0,11,RO,Rondônia,-10.83,-63.34,Norte,1100379,Alto Alegre dos Parecis,-12.13200,-61.8350,0,2,69,America/Porto_Velho
0,11,RO,Rondônia,-10.83,-63.34,Norte,1100403,Alto Paraíso,-9.71429,-63.3188,0,675,69,America/Porto_Velho
0,11,RO,Rondônia,-10.83,-63.34,Norte,1100346,Alvorada D'Oeste,-11.34630,-62.2847,0,35,69,America/Porto_Velho
0,11,RO,Rondônia,-10.83,-63.34,Norte,1100023,Ariquemes,-9.90571,-63.0325,0,7,69,America/Porto_Velho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,52,GO,Goiás,-15.98,-49.86,Centro-Oeste,5222005,Vianópolis,-16.74050,-48.5159,0,9641,62,America/Sao_Paulo
25,52,GO,Goiás,-15.98,-49.86,Centro-Oeste,5222054,Vicentinópolis,-17.73220,-49.8047,0,9657,64,America/Sao_Paulo
25,52,GO,Goiás,-15.98,-49.86,Centro-Oeste,5222203,Vila Boa,-15.03870,-47.0520,0,67,61,America/Sao_Paulo
25,52,GO,Goiás,-15.98,-49.86,Centro-Oeste,5222302,Vila Propício,-15.45420,-48.8819,0,1068,62,America/Sao_Paulo


# Data ingestion

In [12]:
#Creating client to connect to MinIO
client = boto3.client('s3',
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='E3sfE0hkvnUXBcpU',
    aws_secret_access_key='fc2kh6jZjHPyl6opKA8bTYA9xLKw2J8I',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [14]:

for i in range(states.shape[0]):
  csv_buffer = StringIO()
  state_name = states.iloc[i,1]
  cities_from_state = df[df['uf'] == states.iloc[i,1]]
  cities_from_state.to_csv(csv_buffer)
  print(state_name)
  client.put_object(Body=csv_buffer.getvalue(), Bucket='exercicio-aula-06', Key=state_name+"/cidades.csv")
  print('OK!\n')

RO
OK!

AC
OK!

AM
OK!

RR
OK!

PA
OK!

AP
OK!

TO
OK!

MA
OK!

PI
OK!

CE
OK!

RN
OK!

PB
OK!

PE
OK!

AL
OK!

SE
OK!

BA
OK!

MG
OK!

ES
OK!

RJ
OK!

SP
OK!

PR
OK!

SC
OK!

RS
OK!

MS
OK!

MT
OK!

GO
OK!

DF
OK!

